In [1]:
import json
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


In [26]:
file = open('./laureate.json')
laureates = json.load(file)['laureates']
file.close()
file = open('./country.json')
countries = json.load(file)['countries']
file.close()

In [4]:
# result = requests.get('https://api.nobelprize.org/v1/laureate.json')
# laureates = result.json()['laureates']
# laureates

In [28]:
# result = requests.get('http://api.nobelprize.org/v1/country.json')
# country = result.json()
# country

In [29]:
laureates = pd.DataFrame(laureates)
countries = pd.DataFrame(countries)
# df.iloc[1]
countries.iloc[1]

name    Argentina
code           AR
Name: 1, dtype: object

In [30]:
name = []
country = []
diedCountry = []
gender = []
prizeYear = []
prizeCategory = []

for i,item in laureates.iterrows():
    for j in item['prizes']:
        if pd.isna(item['surname']) :
            name.append(item['firstname'])
        else:
            name.append(item['firstname']+' '+item['surname'])
        
        if pd.isna(item['bornCountryCode']):
            country.append('International')
        else:
            country.append(countries[countries['code'] == item['bornCountryCode']].name.iloc[0])

        gender.append(item['gender'])
        prizeYear.append(j['year'])
        prizeCategory.append(j['category'])

df = pd.DataFrame({
    'name':name,
    'country':country,
    'gender':gender,
    'prizeYear':prizeYear,
    'prizeCategory':prizeCategory
}).sort_values(by='prizeYear').reset_index(drop=True)

In [31]:
# laureates[(laureates['firstname']=='Abdulrazak')  & (laureates['surname']=='Gurnah')].bornCountry = 'TN'
# countries[countries['name'] == 'Tanzania']
countArr = []
count = -1
temp = 0
for i in range(len(df)):
    if df['prizeYear'].iloc[i] != temp:
        count = -1
        temp = df['prizeYear'].iloc[i]

    count = count + 1
    countArr.append(count)
df['count'] = countArr

In [8]:
# tempdf[tempdf.bornCountry.isnull().sum()]
# df.country.isnull().sum()
df[:20]

,name,country,gender,prizeYear,prizeCategory,count
0,Wilhelm Conrad Röntgen,Bavaria,male,1901,physics,0
1,Emil von Behring,Free City of Danzig,male,1901,medicine,1
2,Henry Dunant,Switzerland,male,1901,peace,2
3,Frédéric Passy,France,male,1901,peace,3
4,Sully Prudhomme,France,male,1901,literature,4
5,Jacobus H. van 't Hoff,the Netherlands,male,1901,chemistry,5
6,Élie Ducommun,Switzerland,male,1902,peace,0
7,Albert Gobat,Switzerland,male,1902,peace,1
8,Ronald Ross,British India,male,1902,medicine,2
9,Theodor Mommsen,Bavaria,male,1902,literature,3


In [24]:
male = df[df['gender']=='male']
female = df[df['gender']=='female']
org = df[df['gender']=='org']

male =  male.groupby('prizeYear')['count'].count().reset_index()
maleYear = male['prizeYear']
maleCummSum = np.array(male['count']).cumsum()
maleCummSum

female =  female.groupby('prizeYear')['count'].count().reset_index()
tempYear = []
tempCount = []
for i,item in male.iterrows():
    if len(female[female['prizeYear'] == item['prizeYear']]) == 0:
        tempYear.append(item['prizeYear'])
        tempCount.append(0)
    else:
        tempYear.append(female[female['prizeYear'] == item['prizeYear']].iloc[0]['prizeYear'])
        tempCount.append(female[female['prizeYear'] == item['prizeYear']].iloc[0]['count'])
female = pd.DataFrame({'prizeYear':tempYear, 'count':tempCount})
femaleYear = female['prizeYear']
femaleCummSum = np.array(female['count']).cumsum()
femaleCummSum

org =  org.groupby('prizeYear')['count'].count().reset_index()
tempYear = []
tempCount = []
for i,item in male.iterrows():
    if len(org[org['prizeYear'] == item['prizeYear']]) == 0:
        tempYear.append(item['prizeYear'])
        tempCount.append(0)
    else:
        tempYear.append(org[org['prizeYear'] == item['prizeYear']].iloc[0]['prizeYear'])
        tempCount.append(org[org['prizeYear'] == item['prizeYear']].iloc[0]['count'])
org = pd.DataFrame({'prizeYear':tempYear, 'count':tempCount})
orgYear = org['prizeYear']
orgCummSum = np.array(org['count']).cumsum()
orgCummSum

array([ 0,  0,  0,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  4,  4,  4,  5,  5,  5,  7,  7,  7,  7,  7,  7,  7,  8,
        8,  8,  8,  8,  8,  8,  8,  8, 10, 10, 11, 11, 11, 11, 12, 12, 12,
       12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 16,
       16, 16, 16, 16, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 20, 20, 21,
       22, 23, 23, 23, 23, 23, 24, 25, 25, 26, 26, 27, 27, 27, 28, 28])

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=maleYear, y=maleCummSum))
fig.add_trace(go.Scatter(x=orgYear, y=orgCummSum))
fig.add_trace(go.Scatter(x=femaleYear, y=femaleCummSum))

fig.show()

In [9]:
year = df['prizeYear'].unique()
df['count'].iloc[1]

0

In [10]:
for i in year:
    subdf = df[df['prizeYear'] == i]
    count = -1
    for key,item in subdf.iterrows():
        count = count + 1
        df['count'].iloc[key] = count


/tmp/ipykernel_23520/2561105409.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count'].iloc[key] = count


In [43]:
px.scatter(df,x='prizeYear',y='count',color='gender')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'gender=male<br>prizeYear=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': 'male',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'male',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['1901', '1901', '1901', ..., '2021', '2021', '2021'], dtype=object),
              'xaxis': 'x',
              'y': array([ 0,  1,  2, ..., 10, 11, 12]),
              'yaxis': 'y'},
             {'hovertemplate': 'gender=female<br>prizeYear=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': 'female',
              'marker': {'color': '#EF553B', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'female',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['1903', '1905', '1909', '1911', '1926', '1928', '1931', '1935', '1938',
                          '1945', '1946', '1947', '1963', '1964', '1966', '1976', '1976', '1977',
                          '1979', '1982', '1983', '1986', '1988', '1991', '1991', '1992', '1993',
                          '1995', '1996', '1997', '2003', '2004', '2004', '2004', '2007', '2008',
                          '2009', '2009', '2009', '2009', '2009', '2011', '2011', '2011', '2013',
                          '2014', '2014', '2015', '2015', '2018', '2018', '2018', '2018', '2019',
                          '2020', '2020', '2020', '2020', '2021'], dtype=object),
              'xaxis': 'x',
              'y': array([ 0,  2,  3,  1,  2,  3,  3,  0,  4,  6,  2,  3,  8,  1,  1,  1,  2,  8,
                           2,  7,  4,  5, 11,  1,  2,  0,  0,  9,  7, 11,  2,  2,  3,  8,  5,  9,
                           0,  2,  4, 11, 12,  2,  3,  4,  4,  1,  9,  2,  7,  0,  3,  6,  7,  1,
                           3,  4,  5, 11,  2]),
              'yaxis': 'y'},
             {'hovertemplate': 'gender=org<br>prizeYear=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': 'org',
              'marker': {'color': '#00cc96', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'org',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['1904', '1910', '1917', '1938', '1944', '1947', '1947', '1954', '1963',
                          '1963', '1965', '1969', '1977', '1981', '1985', '1988', '1995', '1997',
                          '1999', '2001', '2005', '2006', '2007', '2012', '2013', '2015', '2017',
                          '2020'], dtype=object),
              'xaxis': 'x',
              'y': array([3, 1, 0, 0, 4, 6, 7, 3, 1, 3, 5, 2, 6, 5, 5, 5, 1, 6, 5, 7, 5, 0, 0, 2,
                          3, 1, 1, 2]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'gender'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'prizeYear'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})

In [40]:
p=(len(df[df['gender']=='female'])/len(df))*100
p

6.051282051282051

In [ ]:
p len